Thomas Dougherty

Probability and Statistics for Computer Science


Introduction

    To Do:

    Figure out script to iterate through and cleanup all CSVs

    Store data from all CSVs

    Aggregate data from *all* CSVs into one dataframe
    

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from helper_functions import *

#read CSV for March 2018
services = pd.read_csv('data\\2018_03.csv',index_col=False)
services = services.astype({'date':'datetime64'})
#status column is not needed so that will be dropped
services.drop(labels=['status'], axis=1, inplace=True)
# Amtrak rows are dropped because OTP of Amtrak was not tracked in the dataset
services.drop(services[services['type'] == 'Amtrak'].index, inplace=True)
services = services.astype({'train_id' : 'category',
                'stop_sequence' : 'float16',
                'from' : 'category',
                'from_id': 'category',
                'to' : 'category',
                'to_id': 'category',
                'scheduled_time' : 'datetime64',
                'actual_time' : 'datetime64',
                'delay_minutes' : 'float64',
                'line' : 'category',
                'type' : 'category'})

services = services[services['delay_minutes'].notnull() & services['delay_minutes'] >= 1.0]
services

In [ ]:
 #create new dataframe with data broken down by line

njt_lines=services['line'].unique()
max_delays = pd.Series(helper.get_max_delay(services,njt_lines)).round(3)
avg_delays = pd.Series(helper.get_avg_delay(services,njt_lines)).round(3)
dates = pd.Series(helper.get_delay_date(services,njt_lines))


performance_by_line = pd.DataFrame({'Longest Delay (minutes)': max_delays.values,
                                   'Average Delay (minutes)': avg_delays.values,
                                   'Date of Longest Delay' : dates.values},
                                    index=njt_lines)

performance_by_line

Visualization of On-Time Performance by Rail Line

    notes: Princeton Shuttle is outlier considering it's short trip distance

In [ ]:
ax = performance_by_line.plot.bar(y='Longest Delay (minutes)')
ax

Breakdown by day of the week

In [ ]:
performance_by_line['Date of Longest Delay'].apply(pd.Datetime.dayofweek)